In [24]:
from __future__ import print_function
import matplotlib; matplotlib.use('Agg')
import os, numpy as np, time, sys, argparse
from AB3DMOT_libs.utils import Config, get_subfolder_seq, initialize
from AB3DMOT_libs.io import load_detection, get_saving_dir, get_frame_det, save_results, save_affinity
from scripts.post_processing.combine_trk_cat import combine_trk_cat
from xinshuo_io import mkdir_if_missing, save_txt_file
from xinshuo_miscellaneous import get_timestring, print_log

##### main_per_cat

In [25]:
def main_per_cat(cfg, cat, log, ID_start):

	# get data-cat-split specific path
	result_sha = '%s_%s_%s' % (cfg.det_name, cat, cfg.split)
	det_root = os.path.join('./data', cfg.dataset, 'detection', result_sha)
	subfolder, det_id2str, hw, seq_eval, data_root = get_subfolder_seq(cfg.dataset, cfg.split)
	trk_root = os.path.join(data_root, 'tracking')
	save_dir = os.path.join(cfg.save_root, result_sha + '_H%d' % cfg.num_hypo); mkdir_if_missing(save_dir)

	# create eval dir for each hypothesis 
	eval_dir_dict = dict()
	for index in range(cfg.num_hypo):
		eval_dir_dict[index] = os.path.join(save_dir, 'data_%d' % index); mkdir_if_missing(eval_dir_dict[index]) 		

	# loop every sequence
	seq_count = 0
	total_time, total_frames = 0.0, 0
	for seq_name in seq_eval:
		seq_file = os.path.join(det_root, seq_name+'.txt')
		seq_dets, flag = load_detection(seq_file) 				# load detection
		if not flag: continue									# no detection

		# create folders for saving
		eval_file_dict, save_trk_dir, affinity_dir, affinity_vis = \
			get_saving_dir(eval_dir_dict, seq_name, save_dir, cfg.num_hypo)	

		# initialize tracker
		tracker, frame_list = initialize(cfg, trk_root, save_dir, subfolder, seq_name, cat, ID_start, hw, log)

		# loop over frame
		min_frame, max_frame = int(frame_list[0]), int(frame_list[-1])
		for frame in range(min_frame, max_frame + 1):
			# add an additional frame here to deal with the case that the last frame, although no detection
			# but should output an N x 0 affinity for consistency
			
			# logging
			print_str = 'processing %s %s: %d/%d, %d/%d   \r' % (result_sha, seq_name, seq_count, \
				len(seq_eval), frame, max_frame)
			sys.stdout.write(print_str)
			sys.stdout.flush()

			# tracking by detection
			dets_frame = get_frame_det(seq_dets, frame)
			since = time.time()
			results, affi = tracker.track(dets_frame, frame, seq_name)		
			total_time += time.time() - since

			# saving affinity matrix, between the past frame and current frame
			# e.g., for 000006.npy, it means affinity between frame 5 and 6
			# note that the saved value in affinity can be different in reality because it is between the 
			# original detections and ego-motion compensated predicted tracklets, rather than between the 
			# actual two sets of output tracklets
			save_affi_file = os.path.join(affinity_dir, '%06d.npy' % frame)
			save_affi_vis  = os.path.join(affinity_vis, '%06d.txt' % frame)
			if (affi is not None) and (affi.shape[0] + affi.shape[1] > 0): 
				# save affinity as long as there are tracklets in at least one frame
				np.save(save_affi_file, affi)

				# cannot save for visualization unless both two frames have tracklets
				if affi.shape[0] > 0 and affi.shape[1] > 0:
					save_affinity(affi, save_affi_vis)

			# saving trajectories, loop over each hypothesis
			for hypo in range(cfg.num_hypo):
				save_trk_file = os.path.join(save_trk_dir[hypo], '%06d.txt' % frame)
				save_trk_file = open(save_trk_file, 'w')
				for result_tmp in results[hypo]:				# N x 15
					save_results(result_tmp, save_trk_file, eval_file_dict[hypo], \
						det_id2str, frame, cfg.score_threshold)
				save_trk_file.close()

			total_frames += 1
		seq_count += 1

		for index in range(cfg.num_hypo): 
			eval_file_dict[index].close()
			ID_start = max(ID_start, tracker.ID_count[index])

	print_log('%s, %25s: %4.f seconds for %5d frames or %6.1f FPS, metric is %s = %.2f' % \
		(cfg.dataset, result_sha, total_time, total_frames, total_frames / total_time, \
		tracker.metric, tracker.thres), log=log)
	
	return ID_start


##### notebook

In [26]:
args = {"dataset": "KITTI", "split": "val", "det_name": "pointrcnn"}

In [27]:
config_path = './configs/%s.yml' %args['dataset']
cfg, settings_show = Config(config_path)

In [28]:
cfg.cat_lis

AttributeError: 'EasyDict' object has no attribute 'cat_lis'

In [29]:
time_str = get_timestring()
log = os.path.join(cfg.save_root, 'log/log_%s_%s_%s.txt' % (time_str, cfg.dataset, cfg.split))

In [30]:
log

'./results/KITTI\\log/log_20220722_13h07m32s_KITTI_val.txt'

In [31]:
subfolder, det_id2str, hw, seq_eval, data_root = get_subfolder_seq(cfg.dataset, cfg.split)

In [22]:
cfg.dataset

'KITTI'

In [23]:
print(subfolder, '\n', det_id2str, '\n', hw, '\n', seq_eval, '\n', data_root)

training 
 {1: 'Pedestrian', 2: 'Car', 3: 'Cyclist'} 
 {'image': (375, 1242), 'lidar': (720, 1920)} 
 ['0001', '0006', '0008', '0010', '0012', '0013', '0014', '0015', '0016', '0018', '0019'] 
 C:/Users/Admin/Documents/GitHub/AB3DMOT/data/KITTI


In [25]:
time_str = get_timestring()
log = os.path.join(cfg.save_root, 'log/log_%s_%s_%s.txt' % (time_str, cfg.dataset, cfg.split))
for idx, data in enumerate(settings_show):
    print(data)

# ------------------- General Options -------------------------
description                  : AB3DMOT
seed                         : 0

# --------------- main.py
save_root                    : ./results/KITTI
dataset                      : KITTI      # KITTI
split                        : val        # val, test
det_name                     : pointrcnn  # name of the detector [pointrcnn, pvrcnn]
cat_list                     : ['Car', 'Pedestrian', 'Cyclist']

score_threshold              : -10000     # filter out tracklet with low confidence if necessary, default no filtering here but do it in trk_conf_threshold.py
num_hypo                     : 1          # >1 to allow multi-hypothesis tracking

# --------------- model.py
ego_com                      : true      # turn on only slightly reduce speed but increase a lot for performance
vis                          : false       # only for debug or visualization purpose, will significantly reduce speed
affi_pro                     : true


In [2]:
import numpy as np
data = {}
with open('C:\\calib\\0000.txt', 'r') as f:
    for line in f.readlines():
        print(line)
        line = line.rstrip()
        if len(line) == 0: continue
        key, value = line.split(':',1)
        data[key] = np.array([float(x) for x in value.split()])
        

P0: 7.215377000000e+02 0.000000000000e+00 6.095593000000e+02 0.000000000000e+00 0.000000000000e+00 7.215377000000e+02 1.728540000000e+02 0.000000000000e+00 0.000000000000e+00 0.000000000000e+00 1.000000000000e+00 0.000000000000e+00  

P1: 7.215377000000e+02 0.000000000000e+00 6.095593000000e+02 -3.875744000000e+02 0.000000000000e+00 7.215377000000e+02 1.728540000000e+02 0.000000000000e+00 0.000000000000e+00 0.000000000000e+00 1.000000000000e+00 0.000000000000e+00  

P2: 7.215377000000e+02 0.000000000000e+00 6.095593000000e+02 4.485728000000e+01 0.000000000000e+00 7.215377000000e+02 1.728540000000e+02 2.163791000000e-01 0.000000000000e+00 0.000000000000e+00 1.000000000000e+00 2.745884000000e-03  

P3: 7.215377000000e+02 0.000000000000e+00 6.095593000000e+02 -3.395242000000e+02 0.000000000000e+00 7.215377000000e+02 1.728540000000e+02 2.199936000000e+00 0.000000000000e+00 0.000000000000e+00 1.000000000000e+00 2.729905000000e-03  

R_rect 9.999239000000e-01 9.837760000000e-03 -7.4450480000

ValueError: not enough values to unpack (expected 2, got 1)

In [18]:
with open("G:\내 드라이브\data\AIhub\Auto3D\Training\seoul1_raw\drive_621", 'r') as f:
        for line in f.readlines():
            print(line)
            line = line.rstrip()
            if len(line) == 0: continue
            key, value = line.split(':',1)
            if key != 
            data[key] = np.array([float(x) for x in value.split()])

Number: 1

LidarImageOffset

6992

Angle : LR, UD, Rot

-6.308843,-0.010000,0.000000

TranslationMatrix

0.020000,-0.020000,-0.030000

intrinsicCameraParameter

1067.249617,1070.402406,967.123811,600.618148,-0.156654,0.106250,0.000000,0.000000

--------------------------------------------------------------------------------



In [25]:
import numpy as np

In [35]:
# open kitti calib file

calibs = {}
with open("data/KITTI/mini/training/calib/0000.txt", 'r') as f:
    for line in f.readlines():
        print(line)
        line = line.rstrip()
        if len(line) == 0: continue
        key, value = line.split(':',1)
        calibs[key] = np.array([float(x) for x in value.split()])

P0: 7.215377000000e+02 0.000000000000e+00 6.095593000000e+02 0.000000000000e+00 0.000000000000e+00 7.215377000000e+02 1.728540000000e+02 0.000000000000e+00 0.000000000000e+00 0.000000000000e+00 1.000000000000e+00 0.000000000000e+00  

P1: 7.215377000000e+02 0.000000000000e+00 6.095593000000e+02 -3.875744000000e+02 0.000000000000e+00 7.215377000000e+02 1.728540000000e+02 0.000000000000e+00 0.000000000000e+00 0.000000000000e+00 1.000000000000e+00 0.000000000000e+00  

P2: 7.215377000000e+02 0.000000000000e+00 6.095593000000e+02 4.485728000000e+01 0.000000000000e+00 7.215377000000e+02 1.728540000000e+02 2.163791000000e-01 0.000000000000e+00 0.000000000000e+00 1.000000000000e+00 2.745884000000e-03  

P3: 7.215377000000e+02 0.000000000000e+00 6.095593000000e+02 -3.395242000000e+02 0.000000000000e+00 7.215377000000e+02 1.728540000000e+02 2.199936000000e+00 0.000000000000e+00 0.000000000000e+00 1.000000000000e+00 2.729905000000e-03  

R0_rect: 9.999239000000e-01 9.837760000000e-03 -7.44504800

In [34]:
P = calibs['P2']
P = np.reshape(P, (3,4))

V2

array([7.215377e+02, 0.000000e+00, 6.095593e+02, 4.485728e+01,
       0.000000e+00, 7.215377e+02, 1.728540e+02, 2.163791e-01,
       0.000000e+00, 0.000000e+00, 1.000000e+00, 2.745884e-03])

In [ ]:
P = 